The individual mandatory tour frequency model **predicts the number of work and school tours taken by each person with a mandatory DAP**. The primary drivers of mandatory tour frequency are demographics, accessibility-based parameters such as drive time to work, and household automobile ownership.

In [1]:
import os
import toml
import pandas as pd
import psrc_theme
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots

import plotly.io as pio
# to show plotly figures in quarto HTML file
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color"

# run get_data.ipynb to load validation data
%store -r validation_data

config = toml.load(os.path.join(os.getcwd(),'validation_configuration.toml'))

In [2]:
# read data
land_use = validation_data.land_use.copy()
person = validation_data.persons_data_uncloned.copy()
hh = validation_data.hh_data_uncloned.copy()
tour = validation_data.tours.copy()

In [3]:
hh_data = hh.copy()
per_data = person.copy()
tour_data = tour.copy()

# add auto_ownership with 4+
hh_data['auto_ownership_simple'] = hh_data['auto_ownership'].apply(lambda x: "4+" if x>=4.0 else str(x))

# add person type labels
ptype_cat = {1: "1: Full-Time Worker",
             2: "2: Part-Time Worker",
             3: "3: University Student",
             4: "4: Non-Working Adult Age <65",
             5: "5: Non-Working Adult Age 65+",
             6: "6: High School Student Age 16+",
             7: "7: Child Age 5-15",
             8: "8: Child Age 0-4"}
per_data['ptype_label'] = per_data['ptype'].map(ptype_cat)

# Create distance to work bins: bins of 2 miles up to 46 miles
max_bin = 46
bin_size = 2
per_data['d_work_bin_46mi'] = pd.cut(per_data['distance_to_work'], bins=np.arange(0, max_bin+bin_size, bin_size), labels=[str(i) for i in np.arange(0, max_bin, bin_size)])

# number of mandatory tours by person ID
cdap_reassign = tour_data.groupby(['person_id','tour_category','source'])['tour_id'].size().unstack('tour_category').reset_index()
cdap_reassign['all_tours'] = cdap_reassign[['mandatory','non_mandatory']].sum(axis='columns',skipna=True)

#TODO: check cdap assignment
# reassign cdap for now
cdap_reassign['cdap_reassign'] = cdap_reassign['mandatory'].apply(lambda x: "M" if x>0 else "N")

per_data = per_data.merge(cdap_reassign[['person_id','source','mandatory','cdap_reassign']], how='left', on=['person_id','source'])

per_data = per_data.merge(hh_data[['household_id','auto_ownership','auto_ownership_simple','hh_weight','source']],
                          how='left', on=['household_id','source']) # get auto ownership from hh data

tour_data = tour_data.merge(per_data, how='left', on=['person_id','household_id','source'])

In [4]:
# keep persons with mandatory tours
m_tour_data = pd.DataFrame()
m_per_data = pd.DataFrame()

# compute each source separate;y because of same IDs
for sc in tour_data['source'].unique():

    if sc in ["survey data","unweighted survey"]:
        df_per = per_data[(per_data['source']==sc) & (per_data['cdap_reassign']=="M")].copy()
    else:
        df_per = per_data[(per_data['source']==sc) & (per_data['cdap_activity']=="M")].copy()
    df_tour = tour_data[(tour_data['source']==sc) & (tour_data['tour_category']=="mandatory") & (tour_data['person_id'].isin(df_per['person_id']))].copy()
    df_per = df_per[df_per['person_id'].isin(df_tour['person_id'])].copy()

    # df_tour = df_tour[df_tour['person_id'].isin(df_per['person_id'])].copy()

    m_tour_data = pd.concat([m_tour_data, df_tour])
    m_per_data = pd.concat([m_per_data, df_per])

# total number of persons by source
df_m_person_count = m_per_data.groupby('source')['person_weight'].sum().reset_index()

In [5]:
# total number of worker/student by source
df_worker_count = m_per_data.loc[m_per_data['is_worker']==1].groupby('source')['person_weight'].sum().reset_index()
df_worker_count['worker/student'] = "worker"

df_student_count = m_per_data.loc[m_per_data['is_student']==1].groupby('source')['person_weight'].sum().reset_index()
df_student_count['worker/student'] = "student"

df_worker_student_count = pd.concat([df_worker_count,df_student_count])

# total number of persons by source
df_ptype_count = m_per_data.groupby(['source','ptype_label'])['person_weight'].sum().reset_index()

## mandatory tours per person

$$\text{mandatory tour rates} = {\text{count of all mandatory tours made by M DAP persons} \over \text{M DAP person count}}$$

In [6]:
#| layout-ncol: 2

# mandatory tour rates
df_plot = m_tour_data.groupby(['source','tour_category'])[['tour_weight']].sum().reset_index(). \
    merge(df_m_person_count, how='left', on='source')
df_plot['tour_cat_rate'] = df_plot['tour_weight']/df_plot['person_weight']

fig = px.bar(df_plot, x="tour_category", y="tour_cat_rate", color="source",barmode="group", #template="simple_white",
             # color_discrete_sequence=config['psrc_color'],
             title="mandatory tour rates <br>(persons with mandatory DAP only)")
fig.update_layout(height=400, width=350)
fig.show()

# number of mandatory tours
df_plot = m_tour_data[['person_id','source','tour_category','tour_id','person_weight']].groupby(['person_id','source','tour_category','person_weight']).count().reset_index()
df_plot['num_tours'] = df_plot['tour_id']
df_plot = df_plot.groupby(['source','tour_category','num_tours'])[['person_weight']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['tour_category','source'], group_keys=False)['person_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="num_tours", y="percentage", color="source",barmode="group",
             title="number of mandatory tours in a day <br>(persons with mandatory DAP only)")
fig.update_layout(height=400, width=350, yaxis=dict(tickformat=".0%"))
fig.show()

## work & school tours

- total student/worker count (using `is_worker` and `is_student`, can have intercept)

In [7]:
df_1 = df_worker_student_count.groupby(['source','worker/student'])['person_weight'].sum().reset_index().sort_values('worker/student')
df_1['person_weight'] = df_1['person_weight'].apply(lambda x: '%.2f' % x)
df_1.loc[df_1['source']=="model results"]

,source,worker/student,person_weight
0,model results,student,735830.00
1,model results,worker,1029883.00


In [8]:
# workers
w_tour_data = m_tour_data.loc[m_tour_data['is_worker']==1]
# students
s_tour_data = m_tour_data.loc[m_tour_data['is_student']==1]

In [9]:
# worker tour rates
df_plot_w = w_tour_data.groupby(['source','tour_category'])[['tour_weight']].sum().reset_index(). \
    merge(df_worker_count, how='left', on='source')
df_plot_w['worker/student'] = "worker"
df_plot_s = s_tour_data.groupby(['source','tour_category'])[['tour_weight']].sum().reset_index(). \
    merge(df_student_count, how='left', on='source')
df_plot_s['worker/student'] = "student"

df_plot = pd.concat([df_plot_w,df_plot_s])
df_plot['tour_rate'] = df_plot['tour_weight']/df_plot['person_weight']

fig = px.bar(df_plot, x="worker/student", y="tour_rate", color="source",barmode="group",
             title="mandatory tour rates by worker/student")
fig.update_layout(height=400, width=500)
fig.show()

# fig = px.bar(df_plot, x="tour_category", y="tour_rate", color="source",barmode="group",
#              title="students: mandatory tour rates by tour type")
# fig.update_layout(height=400, width=500)
# fig.show()

In [10]:
# worker tour rates
df_plot = w_tour_data.groupby(['source','tour_type'])[['tour_weight']].sum().reset_index(). \
    merge(df_worker_count, how='left', on='source')
df_plot['tour_cat_rate'] = df_plot['tour_weight']/df_plot['person_weight']

fig = px.bar(df_plot, x="tour_type", y="tour_cat_rate", color="source",barmode="group",
             title="workers: mandatory tour rates by tour type")
fig.update_layout(height=400, width=500)
fig.show()

df_plot = s_tour_data.groupby(['source','tour_type'])[['tour_weight']].sum().reset_index(). \
    merge(df_student_count, how='left', on='source')
df_plot['tour_cat_rate'] = df_plot['tour_weight']/df_plot['person_weight']

fig = px.bar(df_plot, x="tour_type", y="tour_cat_rate", color="source",barmode="group",
             title="students: mandatory tour rates by tour type")
fig.update_layout(height=400, width=500)
fig.show()

- total student/worker count (using `is_worker` and `is_student`, can have intercept)

In [11]:
df_1 = df_worker_student_count.groupby(['source','worker/student'])['person_weight'].sum().reset_index().sort_values('worker/student')
df_1['person_weight'] = df_1['person_weight'].apply(lambda x: '%.2f' % x)
df_1.loc[df_1['source']=="model results"]

,source,worker/student,person_weight
0,model results,student,735830.00
1,model results,worker,1029883.00


In [12]:
m_per_worker = m_per_data.loc[m_per_data['is_worker']==1].copy()
df_plot_w = m_per_worker.groupby(['source','mandatory'])['person_weight'].sum().reset_index()
df_plot_w = df_plot_w.merge(df_worker_count, how='left', on='source')

m_per_worker = m_per_data.loc[m_per_data['is_worker']==1].copy()
df_plot_w = m_per_worker.groupby(['source','mandatory'])['person_weight'].sum().reset_index()
df_plot_w = df_plot_w.merge(df_worker_count, how='left', on='source')

df_plot_w['percentage'] = df_plot_w['person_weight_x']/df_plot_w['person_weight_y']

In [13]:
# number of mandatory tours
df_plot = w_tour_data.groupby(['person_id','source','tour_type','person_weight'])['tour_id'].count().reset_index()

pivoted = df_plot.pivot(index=['person_id','source','person_weight'], columns="tour_type", values="tour_id").fillna(0).reset_index()
df_plot = pivoted.melt(id_vars=['person_id','source','person_weight'], var_name="tour_type")

df_plot['num_tours'] = df_plot['value']
df_plot['ws_weight'] = df_plot['person_weight']
df_plot = df_plot.groupby(['source','tour_type','num_tours'])['ws_weight'].sum().reset_index()
df_plot = df_plot.merge(df_worker_count, how='left', on='source')
df_plot['percentage'] = df_plot['ws_weight']/df_plot['person_weight']

fig = px.bar(df_plot, x="num_tours", y="percentage", color="source",barmode="group",
             facet_col="tour_type",facet_row_spacing=0.2,
             title="workers: number of mandatory tours by tour type")
fig.for_each_annotation(lambda a: a.update(text="tour type="+a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis=dict(tickformat=".2%"))
fig.show()

In [14]:
# number of mandatory tours
df_plot = s_tour_data.groupby(['person_id','source','tour_type','person_weight'])['tour_id'].count().reset_index()

pivoted = df_plot.pivot(index=['person_id','source','person_weight'], columns="tour_type", values="tour_id").fillna(0).reset_index()
df_plot = pivoted.melt(id_vars=['person_id','source','person_weight'], var_name="tour_type")

df_plot['num_tours'] = df_plot['value']
df_plot['ws_weight'] = df_plot['person_weight']
df_plot = df_plot.groupby(['source','tour_type','num_tours'])['ws_weight'].sum().reset_index()
df_plot = df_plot.merge(df_student_count, how='left', on='source')
df_plot['percentage'] = df_plot['ws_weight']/df_plot['person_weight']

fig = px.bar(df_plot, x="num_tours", y="percentage", color="source",barmode="group",
             facet_col="tour_type",facet_row_spacing=0.2,
             title="students: number of mandatory tours by tour type")
fig.for_each_annotation(lambda a: a.update(text="tour type="+a.text.split("=")[-1]))
fig.update_layout(height=400, width=700)
fig.update_yaxes(tickformat=".1%")
fig.show()

In [15]:
df_plot = w_tour_data.groupby(['source','tour_type']).apply(lambda x: pd.Series({
    # tour distance have different variable names in model and survey data
    'agg_tour_dist':(x['tour_distance_one_way'] * x['tour_weight']).sum() + (x['tour_distance'] * x['tour_weight']).sum()
    })).reset_index()
df_plot = df_plot.merge(df_worker_count, how='left', on='source')
df_plot['avg_tour_distance'] = df_plot['agg_tour_dist']/df_plot['person_weight']

fig = px.bar(df_plot, x="tour_type", y="avg_tour_distance", color="source",barmode="group",
            #  facet_col="tour_type",facet_row_spacing=0.2,
             title="workers: average tour distance by tour type")
fig.update_layout(height=400, width=500)
fig.show()

C:\Users\Modeller\AppData\Local\Temp\ipykernel_20208\771498183.py:1: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [16]:
df_plot = s_tour_data.groupby(['source','tour_type']).apply(lambda x: pd.Series({
    # tour distance have different variable names in model and survey data
    'agg_tour_dist':(x['tour_distance_one_way'] * x['tour_weight']).sum() + (x['tour_distance'] * x['tour_weight']).sum()
    })).reset_index()
df_plot = df_plot.merge(df_student_count, how='left', on='source')
df_plot['avg_tour_distance'] = df_plot['agg_tour_dist']/df_plot['person_weight']

fig = px.bar(df_plot, x="tour_type", y="avg_tour_distance", color="source",barmode="group",
            #  facet_col="tour_type",facet_row_spacing=0.2,
             title="students: average tour distance by tour type")
fig.update_layout(height=400, width=500)
fig.show()

C:\Users\Modeller\AppData\Local\Temp\ipykernel_20208\1687638905.py:1: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



- total person count by person type (using `ptype`)

In [17]:
df_1 = m_per_data.groupby(['source','ptype_label'])['person_weight'].sum().reset_index().sort_values('ptype_label')
df_1['person_weight'] = df_1['person_weight'].apply(lambda x: '%.2f' % x)
df_1.loc[df_1['source']=="model results"]

,source,ptype_label,person_weight
0,model results,1: Full-Time Worker,874229.00
1,model results,2: Part-Time Worker,106761.00
2,model results,3: University Student,106969.00
3,model results,6: High School Student Age 16+,120784.00
4,model results,7: Child Age 5-15,390242.00
5,model results,8: Child Age 0-4,117835.00


In [18]:

# mandatory tour rates
df_plot = m_tour_data.groupby(['source','ptype_label','tour_type'])[['tour_weight']].sum().reset_index(). \
    merge(df_ptype_count, how='left', on=['source','ptype_label'])
df_plot['tour_cat_rate'] = df_plot['tour_weight']/df_plot['person_weight']
# df_plot
fig = px.bar(df_plot, x="tour_type", y="tour_cat_rate", color="source",barmode="group",
             facet_col='ptype_label',facet_col_wrap=2,
             title="mandatory tour rates <br>(persons with mandatory DAP only)")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(height=700, width=670)
fig.show()

- total person count by person type (using `ptype`)

In [19]:
df_1 = df_ptype_count.sort_values('ptype_label')
df_1['person_weight'] = df_1['person_weight'].apply(lambda x: '%.2f' % x)
df_1.loc[df_1['source']=="model results"]

,source,ptype_label,person_weight
0,model results,1: Full-Time Worker,874229.00
1,model results,2: Part-Time Worker,106761.00
2,model results,3: University Student,106969.00
3,model results,6: High School Student Age 16+,120784.00
4,model results,7: Child Age 5-15,390242.00
5,model results,8: Child Age 0-4,117835.00


In [20]:
# number of mandatory tours
df_plot = m_tour_data[['person_id','source','ptype_label','tour_type','tour_id','person_weight']].groupby(['person_id','source','ptype_label','tour_type','person_weight']).count().reset_index()

pivoted = df_plot.pivot(index=['person_id','source','ptype_label','person_weight'], columns="tour_type", values="tour_id").fillna(0).reset_index()
df_plot = pivoted.melt(id_vars=['person_id','source','ptype_label','person_weight'], var_name="tour_type")

df_plot['num_tours'] = df_plot['value']
df_plot['ws_weight'] = df_plot['person_weight']
df_plot = df_plot.groupby(['source','ptype_label','tour_type','num_tours'])['ws_weight'].sum().reset_index()
df_plot = df_plot.merge(df_ptype_count, how='left', on=['source','ptype_label'])
df_plot['percentage'] = df_plot['ws_weight']/df_plot['person_weight']

fig = px.bar(df_plot, x="num_tours", y="percentage", color="source",barmode="group",
             facet_row="ptype_label",facet_col="tour_type",
             title="number of work/school tours by person type")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(height=1050, width=700)
fig.update_yaxes(tickformat=".1%")
fig.show()

## number of mandatory tours per person by segment

In [21]:
df_1 = m_per_data.groupby(['source','auto_ownership_simple'])['person_weight'].sum().reset_index()
df_2 = tour_data.loc[tour_data['tour_category']=="mandatory"].groupby(['source','auto_ownership_simple'])['tour_weight'].sum().reset_index()

# plot1
df_plot = df_1.merge(df_2, how='left', on=['source','auto_ownership_simple'])
df_plot['tour_rate'] = df_plot['tour_weight']/df_plot['person_weight']

fig = px.bar(df_plot, x="auto_ownership_simple", y="tour_rate", color="source",barmode="group",
             title="mandatory tour rates by auto ownership (persons with mandatory DAP only)")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700)
fig.update_xaxes(dtick=1)
fig.show()

- auto ownership by student/worker

In [22]:
tour_data['tour_type'].value_counts()

tour_type
work        1042489
shopping     765488
school       734069
othdiscr     629420
escort       575891
othmaint     428608
eatout       293464
eat          223123
social       190207
business      50401
maint         36137
Name: count, dtype: int64

In [23]:
# worker
df_1 = m_per_data.loc[m_per_data['is_worker']==1].groupby(['source','auto_ownership_simple'])['person_weight'].sum().reset_index()
df_2 = tour_data.loc[(tour_data['is_worker']==1) & (tour_data['tour_type']=="work")]. \
    groupby(['source','auto_ownership_simple','tour_type'])['tour_weight'].sum().reset_index()
df_worker_auto = df_1.merge(df_2, how="left", on=['source','auto_ownership_simple'])

df_worker_auto['tour_rate'] = df_worker_auto['tour_weight']/df_worker_auto['person_weight']
df_worker_auto['person_type'] = "worker"

# student
df_1 = m_per_data.loc[m_per_data['is_student']==1].groupby(['source','auto_ownership_simple'])['person_weight'].sum().reset_index()
df_2 = tour_data.loc[(tour_data['is_student']==1) & (tour_data['tour_type']=="school")]. \
    groupby(['source','auto_ownership_simple','tour_type'])['tour_weight'].sum().reset_index()
df_student_auto = df_1.merge(df_2, how="left", on=['source','auto_ownership_simple'])

df_student_auto['tour_rate'] = df_student_auto['tour_weight']/df_student_auto['person_weight']
df_student_auto['person_type'] = "student"

df_tour_auto = pd.concat([df_worker_auto,df_student_auto])

# df_tour_auto

In [24]:
fig = px.bar(df_tour_auto, x="auto_ownership_simple", y="tour_rate", color="source",barmode="group",
             facet_col='person_type',
             title="number of work/school tour rates per worker/student by auto ownership")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700)
fig.update_xaxes(dtick=1)
fig.show()

only 20 samples in unweighted students with auto ownership = 4+

In [25]:
df_1 = m_per_data.groupby(['source','d_work_bin_46mi'])['person_weight'].sum().reset_index()
df_2 = tour_data.loc[tour_data['tour_category']=="mandatory"].groupby(['source','d_work_bin_46mi'])['tour_weight'].sum().reset_index()

# plot1
df_plot = df_1.merge(df_2, how='left', on=['source','d_work_bin_46mi'])
df_plot['tour_rate'] = df_plot['tour_weight']/df_plot['person_weight']


fig = px.line(df_plot, x='d_work_bin_46mi', y="tour_rate", color="source",
               title="mandatory tour rates by distance to work (persons with mandatory DAP only)")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700)
fig.show()

C:\Users\Modeller\AppData\Local\Temp\ipykernel_20208\229107181.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



C:\Users\Modeller\AppData\Local\Temp\ipykernel_20208\229107181.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

